# Freyberg Model PEST setup example
Herein, we will show users how to use pyEMU to setup a groundwater model for use in pest.  We will cover the following topics:
- setup pilot points as parameters, including 1st-order tikhonov regularization
- setup other model inputs as parameters
- setup simulated water levels as observations
- setup simulated water budget components as observations (or forecasts)
- create a pest control file and adjust observation weights to balance the objective function

Note that, in addition to `pyemu`, this notebook relies on `flopy`. `flopy` can be obtained (along with installation instructions) at https://github.com/modflowpy/flopy.

In [ ]:
%matplotlib inline
import os
import shutil
import platform
import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib.patches import Rectangle as rect
import matplotlib.pyplot as plt
import warnings
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
newparams = {'legend.fontsize':10, 'axes.labelsize':10,
             'xtick.labelsize':10, 'ytick.labelsize':10,
             'font.family':'Univers 57 Condensed', 
             'pdf.fonttype':42}
plt.rcParams.update(newparams)
import pyemu
from pathlib import Path
import basic
import flopy
import conda_scripts.utils.folium_maps as fm

Some pathing trickery to make sure we can run this whole nb in a sandbox (and to make sure we dont break paths if we keep running this cell!)

In [ ]:
# egpath = Path(".").absolute()
# while egpath.name != 'examples':
#     os.chdir('..')
#     egpath = Path(".").absolute()

# model_ws = Path("Freyberg_transient").absolute()
# tmp_path = Path("modflow_to_pest_like_a_boss").absolute()

# EXE_DIR = Path("..","bin").absolute()
# if "window" in platform.platform().lower():
#     EXE_DIR = Path(EXE_DIR,"win")
# elif "darwin" in platform.platform().lower() or "macos" in platform.platform().lower():
#     EXE_DIR = Path(EXE_DIR,"mac")
# else:
#     EXE_DIR = Path(EXE_DIR,"linux")
    
# basename = Path(model_ws).name
# new_d = Path(tmp_path, basename)
# if new_d.exists():
#     shutil.rmtree(new_d)
# Path(tmp_path).mkdir(exist_ok=True)
# # creation functionality
# shutil.copytree(model_ws, new_d)

# os.chdir(tmp_path)
# print(Path.cwd())

## Model background
This example is based on the synthetic classroom model of Freyberg(1988).  The  model is a 2-dimensional MODFLOW model with 1 layer,  40 rows, and 20 columns.  The model has 2 stress periods: an initial steady-state stress period used for calibration, and a 5-year transient stress period.  The calibration period uses the recharge and well flux of Freyberg(1988); the last stress period use 25% less recharge and 25% more pumping to represent future conditions for a forecast period.

Freyberg, David L. "AN EXERCISE IN GROUND‐WATER MODEL CALIBRATION AND PREDICTION." Groundwater 26.3 (1988): 350-360.

In [ ]:
ml = basic.load_model()

In [ ]:
# #load the existing model and save it in a new dir and make sure it runs

# model_ws = new_d.relative_to(tmp_path)
# ml = flopy.modflow.Modflow.load("freyberg.nam",model_ws=model_ws,verbose=False)
# ml.model_ws = "temp"
# ml.exe_name = "mfnwt"
# [shutil.copy2(os.path.join(EXE_DIR,f),os.path.join(ml.model_ws,f)) for f in os.listdir(EXE_DIR)]
# ml.write_input()
# pyemu.os_utils.run("mfnwt freyberg.nam", cwd=ml.model_ws)

# Observations
## HOB observations
Here we are going to setup an ``hob`` package to handle getting the observations from modflow.  Normally, you would already have this file made, but here we are just making one for fun

In [ ]:
# hds = flopy.utils.HeadFile(os.path.join(ml.model_ws,"freyberg.hds"))
# rc_df = pd.read_csv(Path(egpath, "Freyberg","misc","obs_rowcol.dat"),delim_whitespace=True)
# data = hds.get_alldata()
# obs = []
# roff = 0.0#ml.dis.delc.array[0] / 2.0
# coff = 0.0#ml.dis.delr.array[0] / 2.0
# for n,r,c in zip(rc_df.name,rc_df.row,rc_df.col):
#     name = "i{1:02d}j{2:02d}".format(n,r-1,c-1)
#     d = np.zeros((data.shape[0]-1,2))
#     d[:,0] = hds.times[1:]
#     d[:,1] = data[1:,0,r-1,c-1] + np.random.randn(d.shape[0]) #add some random noise to the observations
#     obs.append(flopy.modflow.HeadObservation(ml,obsname=name,layer=0,row=r-1,
#                                   column=c-1,roff=roff,coff=coff,
#                                   time_series_data=d))
# flopy.modflow.ModflowHob(ml,obs_data=obs,iuhobsv=600)
# ext_path = os.path.join(ml.model_ws,"ref")
# if os.path.exists(ext_path):
#     shutil.rmtree(ext_path)
# print(ext_path)
# os.mkdir(ext_path)
# ml.external_path = os.path.split(ext_path)[-1]
# ml.upw.hk.fmtin = "(FREE)"
# ml.upw.sy.fmtin = "(FREE)"
# ml.rch.rech.fmtin = "(FREE)"
# ml.write_input()
# pyemu.os_utils.run("mfnwt freyberg.nam", cwd=ml.model_ws)

``pyemu`` has a helper function to setup this instruction file for you and also load observations into dataframe

In [ ]:
# hob_df = pyemu.gw_utils.modflow_hob_to_instruction_file(os.path.join(ml.model_ws,ml.name+".hob.out"))

The dataframe returned has a lot of useful info that we will use later...


In [ ]:
# hob_df.head()

## list file budget components as observations (or forecasts)

Here we will use ``flopy`` and ``pyemu`` to load each of the flux and volume budget components from the ``modflow`` list file to use as observations (or forecasts).  These are valuable pieces of information and since observations are free, why not include them?  This helper function writes two instruction files: ``<flx_filename>.ins`` and ``<vol_filename>.ins``

In [ ]:
# # the flux budget output filename that will be written during each forward run
# flx_filename=os.path.join(ml.model_ws,"flx.out")

# # the volumne budget output filename that will be written during each forward run
# vol_filename = os.path.join(ml.model_ws,"vol.out")
# df_wb = pyemu.gw_utils.setup_mflist_budget_obs(os.path.join(ml.model_ws,ml.name+".list"))

In [ ]:
# df_wb.head()

# Parameters

## pilot points

Here we will setup pilot points for several array-based ``modflow`` inputs using ``pyemu``


### setup pilot point locations

first specify what pilot point names we want to use for each model layer (counting from 0).  Here we will setup pilot points for ``hk``, ``sy`` and ``rech``.  The ``rech`` pilot points will be used as a single multiplier array for all stress periods to account for potential spatial bias in recharge.   

In [ ]:
prefix_dict= {0:["hk1","sy1","ss1"],
             1:["hk2","sy2","ss2"],
             2:["hk3","ss3"]}
ppfolder = 'pilot_points'

This helper function is doing a lot of things: writing templates, pilot point files, and creating a shapefile of pilot points.  The ``every_n_cell`` arg is key: it decides how many cells to skip between pilot point locations - since we passed the ``model``, only active model cells get pilot points (using ``bas6.ibound``).  Like many things with ``flopy``, the ``SpatialReference`` is used to define pilot point ``x`` and ``y `` coordinates

# show map of areas where active pilot points are located

In [ ]:
import copy
ib= copy.deepcopy(ml.bas6.ibound.array[0])

i, j = np.indices(ib.shape)

ib[i<115] = 0
plt.imshow(ib)

In [ ]:
# sr = pyemu.helpers.SpatialReference.from_namfile(os.path.join(ml.model_ws, ml.namefile),
#                                                  delr=ml.dis.delr, delc=ml.dis.delc)
sr = pyemu.helpers.SpatialReference(
                                    delr=ml.dis.delr,
                                    delc=ml.dis.delc,
                                    xll=6296111.0,
                                    yll=1940784.0,
                                    rotation=0.0, 
                                    proj4_str='epsg:2226', 
                                    units='feet',
                                    lenuni=1)
sr

In [ ]:
pp_cells = 7
pp_df = pyemu.pp_utils.setup_pilotpoints_grid(
                                              sr = sr,
                                                ibound = ml.bas6.ibound.array,
                                                # ibound = ib,
                                              prefix_dict=prefix_dict,
                                              every_n_cell=pp_cells,
                                              pp_dir=os.path.join(ml.model_ws, ppfolder),
                                           tpl_dir=os.path.join(ml.model_ws, ppfolder),
                                              shapename=os.path.join(ml.model_ws,ppfolder,"pp.shp"))

The ``dataframe`` return has the same info as the shapefile that was written - useful info, right?

In [ ]:
# pp_df.loc[:,'x'] = ml.modelgrid.get_xcellcenters_for_layer(0)[
#     pp_df.loc[:,'i'].values,pp_df.loc[:,'j'].values]
# pp_df.loc[:,'y'] = ml.modelgrid.get_ycellcenters_for_layer(0)[
#     pp_df.loc[:,'i'].values,pp_df.loc[:,'j'].values]

In [ ]:
tab = gpd.read_file(os.path.join(ml.model_ws, ppfolder, 'pp.shp'))
print(tab.groupby(['pargp']).count().loc[:,['name']])
print(f"total points {tab.shape[0]}")

In [ ]:
tab.crs = 2226

In [ ]:
m = tab.query('k==0').explore(name = 'layer1')
tab.query('k==1').explore(name = 'layer2',m=m)
tab.query('k==2').explore(name = 'layer3',m=m)
fm.add_layers(m)
m

In [ ]:
pp_df.index = pp_df.parnme
pp_df

### geostats and kriging
now that we have pilot points setup, we need to solve the kriging equations for each model cell using pilot point locations.  Since we only have a single set of pilot points that we are reusing for several array-based ``modflow`` inputs, we only need to get the kriging factors once

In [ ]:
# hk_pp = pyemu.pp_utils.pp_file_to_dataframe(os.path.join(ml.model_ws,ppfolder,"hk1pp.dat"))

In [ ]:
# hk_pp.head()

Let's setup a geostatistical structure.  The contribution doesn't matter for pilot point interpolation, but it does matter when we want to form a prior parameter covariance matrix - we will get to that later.  A good rule of thumb is to use an ``a`` value that is three times the pilot point spacing.  Also, since the all of these pilot points will be log transformed, we need to use a log-based geostatistical structure

In [ ]:
a = pp_cells * ml.dis.delr.array[0] * 3.0
v = pyemu.geostats.ExpVario(contribution=1.0,a=a)
gs = pyemu.geostats.GeoStruct(variograms=v,transform="log")
gs.plot()

This is where things get fun.  First we create an ``OrdinaryKrige`` object

In [ ]:
# ok = pyemu.geostats.OrdinaryKrige(geostruct=gs,point_data=hk_pp)

Now we use a helper function to solve the kriging factors for each active model cell: ``OrdinaryKrige.calc_factors_grid()`` includes all the standard kriging arguments, such as search radius, min and max interpolation points,zone_array, as well as the option to save the kriging variance array 

Note: we need to pass out model's spatial reference information. For flopy this used to be contained in model.sr
      However this has been superseded by model.modelgrid. To avoid reliance on a changing (and not always backward
      compatible) code base the sr method has been abstracted into pyemu.

In [ ]:
ml.modelgrid

In [ ]:
ml.bas6.ibound[0].array.shape

In [ ]:
# os.path.join(ml.model_ws, ppfolder)
for lay in [0, 1, 2]:
    
    hk_pp = pyemu.pp_utils.pp_file_to_dataframe(os.path.join(ml.model_ws,ppfolder,f"hk{lay+1}pp.dat"))
    ok = pyemu.geostats.OrdinaryKrige(geostruct=gs,point_data=hk_pp)
    ok.calc_factors_grid(sr, 
                         zone_array=ml.bas6.ibound[lay].array,
                         var_filename=os.path.join(ml.model_ws,ppfolder,f"layer{lay+1}_var.dat"))

    ok.to_grid_factors_file(os.path.join(ml.model_ws,ppfolder,f"pp{lay+1}.fac"))

Ok, we know that this function is slow for bigly models, but it is super convienent and allows a lot of flexibility.  So, once we have calculated the kriging factors for each active model cell, we need to write this to a factors file

Let's check out that kriging variance array....

In [ ]:
var_arr = np.ma.masked_invalid(np.loadtxt(os.path.join(ml.model_ws,ppfolder,"layer1_var.dat")))
fig = plt.figure(figsize=(8,6))
ax = plt.subplot(111,aspect="equal")
ax.pcolormesh(sr.xcentergrid,sr.ycentergrid,var_arr,alpha=0.8)
ax.scatter(hk_pp.x, hk_pp.y,marker='o',s=10, fc = 'k')

var_arr = np.ma.masked_invalid(np.loadtxt(os.path.join(ml.model_ws,ppfolder,"layer2_var.dat")))
fig = plt.figure(figsize=(8,6))
ax = plt.subplot(111,aspect="equal")
ax.pcolormesh(sr.xcentergrid,sr.ycentergrid,var_arr,alpha=0.8)
ax.scatter(hk_pp.x, hk_pp.y,marker='o',s=10, fc = 'k')

In [ ]:
sr.xcentergrid[0,0], sr.ycentergrid[0,0]

In [ ]:
hk_pp.iloc[0,:].values

In [ ]:
# ar  = pyemu.utils.geostats.fac2real(os.path.join(ml.model_ws, ppfolder,'hk1pp.dat'), 
#                               factors_file = os.path.join(ml.model_ws,ppfolder,
#                                                           'pp.fac'), out_file = None)

# fig, ax = basic.map_river(m =ml, )
# ax.pcolormesh(sr.xcentergrid,sr.ycentergrid,np.ma.array(ar,mask =  ar>100),alpha=0.8)
# ax.scatter(hk_pp.x, hk_pp.y,marker='o',s=5, c= 'k')
# plt.savefig('pilot_points_example.png', dpi = 250)


ar  = pyemu.utils.geostats.fac2real(os.path.join(ml.model_ws, ppfolder,'hk3pp.dat'), 
                              factors_file = os.path.join(ml.model_ws,ppfolder,
                                                          'pp3.fac'), out_file = None)

fig, ax = basic.map_river(m =ml, )
ax.pcolormesh(sr.xcentergrid,sr.ycentergrid,np.ma.array(ar,mask =  ar>100),alpha=0.8)
ax.scatter(hk_pp.x, hk_pp.y,marker='o',s=5, c= 'k')
plt.savefig('pilot_points_example.png', dpi = 250)

In [ ]:
# ml2 = flopy.modflow.Modflow.load(ml.namefile, model_ws = 'flopy_error', 
#                                  )

# np.all(ml2.upw.hk.array==0)
# np.all(ml2.upw.vka.array==0)

# pp_file = "hk1pp.dat"
# lay = ''.join(filter(str.isdigit, pp_file.strip('.dat')))
# lay

In [ ]:
ar = basic.load_mod_props(ml)

In [ ]:
out_folder = 'versions\sfr_width_test'

basic.plot_aquifer_props_pilot_points(ml, ppfolder, out_folder, True)

In [ ]:
# def plot_calibrated_props(ml, load_best = True):
#     if load_best:
#         pv = pd.read_csv(os.path.join(ml.model_ws, ppfolder, 'pval.PVAL'), sep = '\s+').rename(columns = {'11':'pval'})
#         pval = pv.rename(lambda x:x.lower()).pval.to_dict()
#     else:
#         pval = ml.mfpar.pval.pval_dict
    
#     hk1= np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'hk1.txt' ))
#     hk2= np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'hk2.txt' ))
#     hk3= np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'hk3.txt' ))
    
#     hk1 = hk1*pval['hk_1']
#     hk2 = hk2*pval['hk_2']
#     hk3 = hk3*pval['hk_3']
    
#     vk1 = hk1*pval['vk_1']
#     vk2 = hk2*pval['vk_2']
#     vk3 = hk3*pval['vk_3']
    
#     ss1 = np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'ss1.txt' ))
#     ss2 = np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'ss2.txt' ))
#     ss3 = np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'ss3.txt' ))
#     ss1 = ss1* pval['ss_1']
#     ss2 = ss2* pval['ss_2']
#     ss3 = ss3* pval['ss_3']
    
#     sy1 = np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'sy1.txt' )) *  pval['sy_1']
#     sy2 = np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'sy2.txt' )) *  pval['sy_2']
    
    
#     return {'hk1':hk1,'hk2':hk2, 'hk3':hk3,
#             'vk1':vk1, 'vk2':vk2, 'vk3':vk3,
#             'ss1':ss1, 'ss2':ss2, 'ss3':ss3,
#             'sy1':sy1, 'sy2':sy2,}, pval

# ppar, pval  = plot_calibrated_props(ml)

# basic.plot_aquifer_prop(ml, np.stack([ppar['hk1'], ppar['hk2'], ppar['hk3'], ]), vmax = .1)
# plt.savefig(os.path.join(ml.model_ws,ppfolder, 'pp_hk.png'), dpi = 250)
# plt.savefig(os.path.join(out_folder, 'hk.png'), dpi = 250)
# basic.plot_aquifer_prop(ml, np.stack([ppar['vk1'], ppar['vk2'], ppar['vk3'], ]), vmin = 1e-7, vmax = .01, title='Vertical Conductivity')
# plt.savefig(os.path.join(ml.model_ws,ppfolder, 'pp_vk.png'), dpi = 250)
# plt.savefig(os.path.join(out_folder, 'vk.png'), dpi = 250)
# basic.plot_aquifer_prop(ml, np.stack([ppar['ss1'], ppar['ss2'], ppar['ss3'], ]), vmin = 1e-6, vmax = 0.01, title='Specific Storage')
# plt.savefig(os.path.join(ml.model_ws,ppfolder, 'pp_ss.png'), dpi = 250)
# plt.savefig(os.path.join(out_folder, 'ss.png'), dpi = 250)
# basic.plot_aquifer_prop(ml, np.stack([ppar['sy1'], ppar['sy2'], ppar['sy2']*0, ]), vmin = .01, vmax = .3, title='Specific Yield')
# plt.savefig(os.path.join(ml.model_ws,ppfolder, 'pp_sy.png'), dpi = 250)# basic.plot_aquifer_prop(ml, np.stack([ppar['hk1'], ar['hk1'], ar['hk1'], ]), vmax = .01)
# plt.savefig(os.path.join(out_folder, 'sy.png'), dpi = 250)

In [ ]:
# def load_mod_props(ml, load_best = True):
#     if load_best:
#         pv = pd.read_csv(os.path.join(ml.model_ws, ppfolder, 'pval.PVAL'), sep = '\s+').rename(columns = {'11':'pval'})
#         pval = pv.rename(lambda x:x.lower()).pval.to_dict()
#     else:
#         pval = ml.mfpar.pval.pval_dict
    
#     hk1= np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'hklay1_thck.txt' ), delimiter = ',')
#     hk2= np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'hklay2_thck.txt' ), delimiter = ',')
#     hk3= np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'hklay3_thck.txt' ), delimiter = ',')
    
#     hk1 = hk1*pval['hk_1']
#     hk2 = hk2*pval['hk_2']
#     hk3 = hk3*pval['hk_3']
    
#     vk1 = hk1*pval['vk_1']
#     vk2 = hk2*pval['vk_2']
#     vk3 = hk3*pval['vk_3']
    
#     ss1 = np.ones((ml.dis.nrow, ml.dis.ncol)) *  pval['ss_1']
#     ss2 = np.ones((ml.dis.nrow, ml.dis.ncol)) *  pval['ss_2']
#     ss3 = np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'sslay3_thck.txt' ), delimiter = ',')
#     ss3 = ss3* pval['ss_3']
    
#     sy1 = np.ones((ml.dis.nrow, ml.dis.ncol)) *  pval['sy_1']
#     sy2 = np.ones((ml.dis.nrow, ml.dis.ncol)) *  pval['sy_2']
    
    
#     return {'hk1':hk1,'hk2':hk2, 'hk3':hk3,
#             'vk1':vk1, 'vk2':vk2, 'vk3':vk3,
#             'ss1':ss1, 'ss2':ss2, 'ss3':ss3,
#             'sy1':sy1, 'sy2':sy2,}, pval

# ar, pval  = load_mod_props(ml)

# basic.plot_aquifer_prop(ml, np.stack([ar['hk1'], ar['hk2'], ar['hk3'], ]), vmax = .1)
# plt.savefig(os.path.join(ml.model_ws,ppfolder, 'calib_hk.png'), dpi = 250)
# basic.plot_aquifer_prop(ml, np.stack([ar['vk1'], ar['vk2'], ar['vk3'], ]), vmin = 1e-5, vmax = .01, title='Vertical Conductivity')
# plt.savefig(os.path.join(ml.model_ws,ppfolder, 'calib_vk.png'), dpi = 250)
# basic.plot_aquifer_prop(ml, np.stack([ar['ss1'], ar['ss2'], ar['ss3'], ]), vmin = 1e-5, vmax = .01, title='Specific Storage')
# plt.savefig(os.path.join(ml.model_ws,ppfolder, 'calib_ss.png'), dpi = 250)
# basic.plot_aquifer_prop(ml, np.stack([ar['sy1'], ar['sy2'], ar['sy2']*0, ]), vmin = 1e-5, vmax = .01, title='Specific Yield')
# plt.savefig(os.path.join(ml.model_ws,ppfolder, 'calib_sy.png'), dpi = 250)# basic.plot_aquifer_prop(ml, np.stack([ar['hk1'], ar['hk1'], ar['hk1'], ]), vmax = .01)

# set the pilot point values from the aquifer property arrays


In [ ]:
for group, dfi in pp_df.groupby('pargp'):
    # print(group)
    pp_df.loc[dfi.index, 'parval1'] = ar[group][dfi.loc[:,'i'],dfi.loc[:,'j']]
    
    if (pp_df.loc[dfi.index, 'parval1']==0).sum()>0:
        
        
        zeros = pp_df.loc[(pp_df.loc[dfi.index, 'parval1']==0.0),:].index
        warnings.warn( f"there are {len(zeros)} zeros in {group}. These will be filled with the group average")
        
        print(zeros)
        
        # pp_df.loc[zeros,'parval1'] = pp_df.loc[dfi.index, 'parval1'].mean() 

In [ ]:
pp_df.loc[pp_df.parnme.str.startswith('ss3'),'parval1'] = pp_df.loc[pp_df.parnme.str.startswith('ss2'),'parval1'].mean()

In [ ]:
pp_df.loc[pp_df.parnme.str.startswith('ss3'),'parval1']

In [ ]:
np.unique(ar['ss3'].reshape((-1,)))

## other inputs as parameters

Since we rarely know any model inputs perfectly, it is advisable to subject them to adjustment...not to get a good ``fit``, but so we can account for there contribution to uncertainty...How about the conductance between the surface water and groundwater systems.  In this model, we are using ``drain`` type boundaries.  So, let's setup a multiplier parameter for each ``drain`` cell's conductance.  

Since we told ``flopy`` to write external files, all of the list-type ``modflow`` inputs are also external, which makes this so much easier!  The first thing to do is copy the orginal drain list files (and all other files in the external directory) to a safe place:


In [ ]:
# ext_path = os.path.join(ml.model_ws,"ref")
# ext_files = [f for f in os.listdir(ext_path)]
# drain_files = [f for f in ext_files if "drn" in f.lower()]
# #print(drain_files)
# assert len(drain_files) == ml.nper,"{0},{1}".format(len(drain_files),ml.nper)
# bak_path = os.path.join(ml.model_ws,"bak")
# if os.path.exists(bak_path):
#     shutil.rmtree(bak_path)
# os.mkdir(bak_path)
# for f in ext_files:
#     shutil.copy2(os.path.join(ext_path,f),os.path.join(bak_path,f))
# #assert len(os.listdir(bak_path)) == ml.nper

Now all we need to do is write a template file.  We will also write a generic cooresponding input file that will make testing easier later

In [ ]:
# drn_df = pd.read_csv(os.path.join(bak_path,drain_files[0]),
#                      header=None,names=["l","r","c","stage","cond"],
#                     delim_whitespace=True)
# f_tpl = open(os.path.join(ml.model_ws,"drain_mlt.dat.tpl"),'w')
# f_in = open(os.path.join(ml.model_ws,"drain_mlt.dat"),'w')
# f_tpl.write("ptf ~\n")
# #build parameter names from model cell info
# drn_df.loc[:,"parnme"] = drn_df.apply(lambda x: "drn_i{1:02.0f}j{2:02.0f}".format(x.l-1,x.r-1,x.c-1),axis=1)
# for parnme in drn_df.parnme:
#     f_tpl.write("{0}  ~   {0}   ~\n".format(parnme))
#     f_in.write("{0}     1.0\n".format(parnme))
# f_tpl.close()
# f_in.close()

# Building the pest control file...Finally!

Here we will use the template and instruction files to construct a control file.  Then we will use some ``pandas`` magic to set the appropriate parameter and observation info

In [ ]:
tpl_files = ['pest\dataset11a_with_geo.txt.tpl', 
             'pest\dataset11a_with_geo_2015.txt.tpl',
             'pest\dataset11a_with_geo_2016.txt.tpl',
             'pest\pval.PVAL.tpl']
tpl_files = [os.path.join(ml.model_ws,f) for f in tpl_files]
input_files = ['inputs\dataset11a_with_geo.txt',
               'inputs\dataset11a_with_geo_2015.txt',
               'inputs\dataset11a_with_geo_2016.txt',
               'pval.PVAL']
input_files = [os.path.join(ml.model_ws,f) for f in input_files]

In [ ]:
pp_tpl_files = [os.path.join(ml.model_ws,ppfolder,f) for f in 
                os.listdir(os.path.join(ml.model_ws, ppfolder)) if f.endswith(".tpl")]

pp_input_files = [f.replace(".tpl",'') for f in pp_tpl_files]
pp_tpl_files

In [ ]:
tpl_files.extend(pp_tpl_files)
input_files.extend(pp_input_files)

See why it is important to use a consistent naming structure for the templates-input file pairs?  Its the same for the instruction files

In [ ]:
input_files

In [ ]:
tpl_files

In [ ]:
# ins_files = [os.path.join(ml.model_ws,f) for f in os.listdir(ml.model_ws) if f.endswith(".ins")]
# output_files = [f.replace(".ins",'') for f in ins_files]
# ins_files

ins_files = ['Results\hobs.out.ins']
ins_files = [os.path.join(ml.model_ws,f) for f in ins_files]
output_files =  ['Results\hobs.out']
output_files = [os.path.join(ml.model_ws,f) for f in output_files]

Now use these files to get a ``pyemu.Pst`` instance.  This object has lots of cool functionality...

In [ ]:
pst = pyemu.Pst.from_io_files(tpl_files,input_files,ins_files,output_files)

pst.model_input_data = pst.model_input_data.applymap(lambda x:x.replace("RR_2022\\", ''))
pst.model_output_data = pst.model_output_data.applymap(lambda x:x.replace("RR_2022\\", ''))
display(pst.model_input_data)

In [ ]:
pst.model_output_data

Let's look at some of the important parts of the ``Pst`` class.  First, all attributes coorespond to the names in list in the pest manual.  For instance, the ``* parameter data`` section of the control file is a ``pandas.DataFrame`` attribute named ``parameter_data``:

In [ ]:
pst.parameter_data.head()

We see that the columns of the ``DataFrame`` follow the pest naming conventions.  Its the same for ``* observation data``:

In [ ]:
pst.observation_data.head()

What ``pyemu`` has set as the ``obsval`` is the simulated equivalent, if it is available - in the ``pst_from_io_files()`` helper, ``pyemu`` tries to call ``inschek``, and, if successful, loads the output files from ``inschek``.  This can be very handy for error checking in the forward-run process. However, we still need to get the actual observed data into ``obsval``...remember that dataframe from hob processing?

Notice the ``obsval`` column?  Let's just set the index of this dataframe to ``obsnme``, then pandas does the hard work for us:

In [ ]:
pst.observation_data

In [ ]:
hob_df = pyemu.gw_utils.modflow_hob_to_instruction_file(
    os.path.join(ml.model_ws,'Results','hobs.out'))

hob_df.index = hob_df.obsnme
hob_df.head()

In [ ]:
pst.observation_data.loc[hob_df.index,"obsval"] = hob_df.obsval
pst.observation_data.loc[hob_df.index,:].head()

In [ ]:
pst_temp = pyemu.Pst(os.path.join(ml.model_ws,"pp_pest.pst"))
pst_temp.parrep(r"RR_2022\pp_pest.par")

In [ ]:
pst_temp = pyemu.Pst(os.path.join(ml.model_ws,"pst_temp.pst"))

In [ ]:
pst.observation_data = pst_temp.observation_data

In [ ]:
pst.observation_data.shape

In [ ]:
pst.observation_data

BOOM!  that was easy...trying doing that without pandas....not fun!

We still have a few more items to set to specific values. The biggest one is initial values for parameters - they are given default values of 1.0:

In [ ]:
pst.parameter_data.head()

In [ ]:
par = pst.parameter_data
par.loc[pp_df.index,"parval1"] = pp_df.parval1


In [ ]:
_df = pyemu.pp_utils.pp_tpl_to_dataframe(os.path.join(ml.model_ws, ppfolder,"hk1pp.dat.tpl"))
display(_df.head())

cov = gs.covariance_matrix(x=_df.x, y=_df.y, names=_df.parnme)

# just for a bit of eye-candy; bright yellow indicates higher covariance.
c = plt.imshow(cov.x)
plt.colorbar(c)

Now, we can assign prior information equations for preferred difference. Note that the preferred difference = 0, which means our preferred difference regularization is really a preferred *homogeneity* condition! If observation data don't say otherwise, parameters which are close together should be similar to each other.

The weights on the prior information equations are the Pearson correlation coefficients implied by the covariance matrix.

In [ ]:
# then assign cov pror
pyemu.helpers.first_order_pearson_tikhonov(pst, 
                                            cov=cov,     # the covariance matrix; these can be for some OR all parameters in pst
                                            reset=False, # so as to have both prefered value and prefered difference eqs
                                            abs_drop_tol=0.01) # drop pi eqs that have small weight

In [ ]:
# note the additional number of prior information equations
pst.prior_information.tail()

In [ ]:
# for name, gdf in pp_df.query('i<=115').groupby('pargp'):
#     print(name+'\n', gdf,'----------\n'*5)

In [ ]:
# set pp above wohler to tied
par.loc[pp_df.query('i<=115').index, 'partrans'] = 'tied'

# # add a new column named "partied" and assign the parameter name to which to tie "special_par2"
par.loc[:,'partied'] = np.nan

# go through each group and assign one as the movable parameter and others tied to it
for name, gdf in pp_df.query('i<=115').groupby('pargp'):
    first = gdf.index[0]
    par.loc[gdf.index, 'partied'] = first
    # par.loc[gdf.drop(first).index, 'parval1'] = 1.0
    par.loc[first,'partrans'] = 'log'
    par.loc[first,'partied'] = np.nan
    
# # display for comparison; see partrans and partied columns
# par.loc[par['pargp'] == 'pargp', ['partrans', 'partied']]

In [ ]:
pst_temp.parameter_data.tail()

In [ ]:
par.loc[['ss_1', 'ss_2','ss_3','sy_1', 'sy_2', 'hk_1', 'hk_2', 'hk_3'],'partrans'] = 'fixed'

# par.loc[['vk_1', 'vk_2','vk_3'],'parval1'] =[ pval['vk_1'], pval['vk_2'], pval['vk_3']]
par.loc[['vk_1', 'vk_2','vk_3'],'parval1'] = pst_temp.parameter_data.loc[['vk_1','vk_2','vk_3'], 'parval1']



In [ ]:
# fix swrk

In [ ]:
swr = ['swrk','swrk2015', 'swrk2016']
par.loc[swr, 'parval1'] = pst_temp.parameter_data.loc[swr, 'parval1']
par.loc[swr, 'partrans'] = 'fixed'

In [ ]:
par.partrans.value_counts()

In [ ]:
par.loc[pst_temp.parameter_data.index, 'pargp'] = pst_temp.parameter_data.loc[:, 'pargp'].apply(lambda x: 'pval_'+x).fillna('pargp')

In [ ]:
par = par.sort_values('pargp')

In [ ]:
par.partied.isnull().sum()

Luckily, ``pandas`` makes this very easy.  For example, let's set the ``DRN`` conductance parameters to have initial values of mean of the values in the model currently:


In [ ]:
(par.loc[:,'parval1']==0).sum()

In [ ]:
pst.para

In [ ]:
tab.loc[tab.loc[:,'parnme'].isin(par.loc[(par.loc[:,'parval1']==0)].index)].explore()

In [ ]:
tab

In [ ]:
# avg = ml.drn.stress_period_data[0]["cond"].mean()
# par = pst.parameter_data #just a pointer to the full, long-named attribute
# drn_pars = par.loc[par.parnme.apply(lambda x: x.startswith("drn")),"parnme"].values
# par.loc[drn_pars,"parval1"] = avg
# #set the par group to mean something
# par.loc[drn_pars,"pargp"] = "drn_cond"
# par.loc[drn_pars,"parubnd"] = avg * 10.0
# par.loc[drn_pars,"parlbnd"] = avg * 0.1

Let's set the ``pargp`` for the remaining parameters using that cool pilot point dataframe from eariler...

In [ ]:
par

In [ ]:
par.loc[pp_df.index,"pargp"] = pp_df.loc[:,'pargp']

We need to reset the model run command:

In [ ]:
pst_temp.model_command

In [ ]:
pst.model_command

that is just a generic command. I prefer to use python scripts for this:

In [ ]:
# pst.model_command = ["python pest\forward_run_pp.py"]
pst.model_command = pst_temp.model_command

Let's save this version of the control file

But this means we need to write ``forward_run.py`` and it needs to perform several actions:
- apply kriging factors (using ``pyemu.gw_utils.fac2real()``)
- apply the drain multipliers
- call ``MODFLOW``
- process the ``MODFLOW`` list file

Lucky for you, I already made this file....

In [ ]:
# shutil.copy2(os.path.join("Freyberg_transient","forward_run.py"),os.path.join(ml.model_ws,"forward_run.py"))

### adding prior information

``pyemu`` supports both zero-order (preferred value) and first-order (preferred difference) Tikhonov regularization.  Let's set preferred value for the conductance parameters:

In [ ]:
# pyemu.utils.helpers.zero_order_tikhonov(pst,par_groups=["drn_cond"])
# pst.prior_information.head()

Now, let's set preferred difference equations for pilot point groups.  We will use the Pearson coef as the weight...

In [ ]:
# pp_groups = pp_df.groupby("pargp").groups
# for pargp,par_names in pp_groups.items():
#     this_pp_df = pp_df.loc[par_names,:]
#     cov = gs.covariance_matrix(this_pp_df.x,this_pp_df.y,this_pp_df.parnme)
#     pyemu.helpers.first_order_pearson_tikhonov(pst,cov,reset=False,abs_drop_tol=0.2)

In [ ]:
# pst.prior_information

In [ ]:
# pst.control_data.pestmode = "regularization"

### setting PEST++ options

Some things I like to add:

In [ ]:
pst.pestpp_options["svd_pack"] = "redsvd"
#pst.pestpp_options["forecasts"] = 

### saving the new control file

In [ ]:
pst.parameter_data = par

In [ ]:
pst.write(os.path.join(ml.model_ws,'ies.pst'))

In [ ]:
par.loc[:,['pargp','partrans']].groupby('pargp').value_counts(subset = 'partrans').unstack().fillna(0).sort_index()

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize = (10,10))
ax = sns.stripplot(data=par.query("parval1!=1. ").loc[:,['pargp','parval1']], x="pargp", y="parval1")
ax.set_yscale('log')
ax.grid(True)

labels = ax.get_xticklabels()
ax.set_xticklabels(labels=labels,rotation=45);

In [ ]:
pval

In [ ]:
pvalpest

In [ ]:
def load_mod_props(ml, load_best = True):
    if load_best:
        pv = pd.read_csv(os.path.join(ml.model_ws, ppfolder, 'pval.PVAL'), sep = '\s+').rename(columns = {'11':'pval'})
        pval = pv.rename(lambda x:x.lower()).pval.to_dict()
    else:
        pv = pd.read_csv(os.path.join(ml.model_ws, 'pval.PVAL'), sep = '\s+').rename(columns = {'11':'pval'})
        pval = pv.rename(lambda x:x.lower()).pval.to_dict()
        # pval = ml.mfpar.pval.pval_dict
    
    hk1= np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'hk1.txt' ), )
    hk2= np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'hk2.txt' ), )
    hk3= np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'hk3.txt' ), )
    
    hk1 = hk1*pval['hk_1']
    hk2 = hk2*pval['hk_2']
    hk3 = hk3*pval['hk_3']
    
    vk1 = hk1*pval['vk_1']
    vk2 = hk2*pval['vk_2']
    vk3 = hk3*pval['vk_3']
    
    ss1 = np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'ss1.txt' ),)
    ss2 = np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'ss2.txt' ),)
    ss3 = np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'ss3.txt' ),)
    ss3 = ss3* pval['ss_3']
    
    sy1 = np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'sy1.txt' ),)
    sy2 = np.genfromtxt(os.path.join(ml.model_ws, ppfolder, 'sy2.txt' ),)
    
    
    return {'hk1':hk1,'hk2':hk2, 'hk3':hk3,
            'vk1':vk1, 'vk2':vk2, 'vk3':vk3,
            'ss1':ss1, 'ss2':ss2, 'ss3':ss3,
            'sy1':sy1, 'sy2':sy2,}, pval

arpest, pvalpest  = load_mod_props(ml, False)

basic.plot_aquifer_prop(ml, np.stack([arpest['hk1'], arpest['hk2'], arpest['hk3'], ]), vmax = .1)
plt.savefig(os.path.join(ml.model_ws,ppfolder, 'pphk.png'), dpi = 250)
basic.plot_aquifer_prop(ml, np.stack([arpest['vk1'], arpest['vk2'], arpest['vk3'], ]), vmin = 1e-5, vmax = .01, title='Vertical Conductivity')
plt.savefig(os.path.join(ml.model_ws,ppfolder, 'ppvk.png'), dpi = 250)
basic.plot_aquifer_prop(ml, np.stack([arpest['ss1'], arpest['ss2'], arpest['ss3'], ]), vmin = 1e-5, vmax = .01, title='Specific Storage')
plt.savefig(os.path.join(ml.model_ws,ppfolder, 'ppss.png'), dpi = 250)
basic.plot_aquifer_prop(ml, np.stack([arpest['sy1'], arpest['sy2'], arpest['sy2']*0, ]), vmin = 1e-5, vmax = .01, title='Specific Yield')
plt.savefig(os.path.join(ml.model_ws,ppfolder, 'ppsy.png'), dpi = 250)
# basic.plot_aquifer_prop(ml, np.stack([ar['hk1'], ar['hk1'], ar['hk1'], ]), vmax = .01)

In [ ]:
pst.write_input_files('RR_2022')